# Práctica 9 - Embedding LSTM

> **Intituto Politécnico Nacional**
>
> **Centro de Investigación en Computación**
>
> 
> 
> Departamento de Diplomados y Extensión Profesional
> 
> Diplomado en Inteligencia Artificial
> 
> * Módulo 11 - Parte II
> 
> Profesor: Alan Badillo Salas (badillosalas@outlook.com)

In [ ]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

In [ ]:
text_only_train_ds = train_ds.map(lambda x, y: x)

In [ ]:
max_length = 600
max_tokens = 500

text_vectorization = keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)

text_vectorization.adapt(text_only_train_ds)

In [ ]:
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")

embedded = keras.layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)

x = keras.layers.Bidirectional(keras.layers.LSTM(32))(embedded)
x = keras.layers.Dropout(0.5)(x)

outputs = keras.layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(
    optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("resultados/embeddings_bidir_gru.keras", save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=1, callbacks=callbacks)

In [ ]:
model = keras.models.load_model("resultados/embeddings_bidir_gru.keras", safe_mode=False)
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")